Importar librerias de python.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

Importar csv para trabajar.

In [2]:
#Importar csv a trabajar. 
df1 = pd.read_csv('../P1_ML/MLOpsReviews/amazon_prime_titles.csv')
df_amazon = pd.DataFrame(df1)

df2 = pd.read_csv('../P1_ML/MLOpsReviews/disney_plus_titles.csv')
df_disney = pd.DataFrame(df2)

df3 = pd.read_csv('../P1_ML/MLOpsReviews/hulu_titles.csv')
df_hulu = pd.DataFrame(df3)

df4 = pd.read_csv('../P1_ML/MLOpsReviews/netflix_titles.csv')
df_netflix = pd.DataFrame(df4)

Luego de hacer exploración a los datos, unifico el tipo de dato de cada df, para que se puedan trabajar en conjunto.

In [3]:
df_hulu['cast'] = df_hulu['cast'].astype('object') #La columna df_hulu['cast'] tenía un tipo de dato diferente. Unificar el tipo de dato.

In [4]:
#Creación de id para cada plataforma.
df_amazon['show_id'] = 'as' + pd.RangeIndex(1, len(df_amazon)+1).astype(str)
df_disney['show_id'] = 'ds' + pd.RangeIndex(1, len(df_disney)+1).astype(str)
df_hulu['show_id'] = 'hs' + pd.RangeIndex(1, len(df_hulu)+1).astype(str)
df_netflix['show_id'] = 'ns' + pd.RangeIndex(1, len(df_netflix)+1).astype(str)

In [5]:
df_movies = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True).drop_duplicates() #Concatenar df para trabajar en conjunto.
df_movies.rename(columns = {'show_id' : 'id'}, inplace = True) #Renombrar columna id

In [6]:
#Crear una función para explorar los datos. Nulos, registros unicos y relación entre los mismos.
def registros(columna):
    nulos = df_movies[columna].isnull().sum()
    elements = []
    count_unic = 0
    count_g = 0
    for i in df_movies[columna]:
        count_g += 1
        elements.append(i)
    elements = set(elements)
    for e in elements:
        count_unic += 1
    return 'La cantidad de registros nulos para la columna {} es igual a: {}, con un total de {} datos unicos y registros repetidos entre ellos de {}'.format(columna, nulos, count_unic, count_g-count_unic)
for columna in df_movies.columns:
    print(registros(columna))

La cantidad de registros nulos para la columna id es igual a: 0, con un total de 22998 datos unicos y registros repetidos entre ellos de 0
La cantidad de registros nulos para la columna type es igual a: 0, con un total de 2 datos unicos y registros repetidos entre ellos de 22996
La cantidad de registros nulos para la columna title es igual a: 0, con un total de 22115 datos unicos y registros repetidos entre ellos de 883
La cantidad de registros nulos para la columna director es igual a: 8259, con un total de 10107 datos unicos y registros repetidos entre ellos de 12891
La cantidad de registros nulos para la columna cast es igual a: 5321, con un total de 16749 datos unicos y registros repetidos entre ellos de 6249
La cantidad de registros nulos para la columna country es igual a: 11499, con un total de 887 datos unicos y registros repetidos entre ellos de 22111
La cantidad de registros nulos para la columna date_added es igual a: 9554, con un total de 2059 datos unicos y registros repet

In [7]:
df_movies.drop(['director', 'country', 'description'], axis = 1, inplace = True)#Eliminar columnas que no agreguen valor a la consulta.

A la columna cast, de la cual no podemos obtener una relacion con las demás columnas para llenar nulos y tampoco eliminar porque vamos a trabajar sobre los actores, aplicaremos la funcion fillna con: 'Not info'.

In [8]:
df_movies['cast'].fillna('Not info', inplace = True)

Vamos a trabajar primero sobre la columna date para obtener el año y disminuir los nulos en el df.

In [9]:
#Separar los datos de la columna added para concatenar la fecha en el formato solicitado.
fecha = df_movies['date_added'].str.split(expand=True)
fecha.columns = ['month', 'day', 'year']
df_movies = pd.concat([df_movies, fecha], axis=1)
#Crear la columna con los días, y eliminar la columna original que tiene el caracter ','.
df_day = df_movies['day'].str.split(',', expand=True)
df_movies['n_day'] = df_day[0]
df_movies.drop(['day'], axis = 1, inplace = True)

In [10]:
#Verificar los nombres de los meses para corregirlos y poder aplicar formato fecha de python en la columna.
meses = [i for i in df_movies.month]
meses_totales = list(set(meses))
for e in meses_totales:
    if type(e) != str:
        meses_totales.remove(e)
meses_totales

['April',
 'March',
 'January',
 'December',
 'July',
 'June',
 'October',
 'November',
 'August',
 'May',
 'February',
 'September']

In [11]:
#Cambiar nombres de los meses.
df_movies['month'] = df_movies['month'].replace({ 'January': 'Jan', 
                                                 'February': 'Feb', 
                                                 'March': 'Mar', 
                                                 'April': 'Apr', 
                                                 'May': 'May', 
                                                 'June': 'Jun', 
                                                 'July': 'Jul', 
                                                 'August': 'Aug', 
                                                 'September': 'Sept', 
                                                 'October': 'Oct', 
                                                 'November' : 'Nov',
                                                 'December': 'Dec'  })

Hallar la media aritmética y la moda del date added, para poder rellenar los campos nulos y tener un acercamiento al año. Para los meses y días podemos a usar Jan/01.

In [12]:
df_movies_str = pd.DataFrame(df_movies)
median = df_movies_str[(df_movies_str.year.notnull())]['year'].median()
moda = df_movies_str[(df_movies_str.year.notnull())]['year'].mode()
print(f'La mediana es {int(median)} y la moda {moda[0]}')

La mediana es 2019 y la moda 2019


Llenamos los valores nulos de la columna year con la mediana/moda que es 2019.

In [13]:
df_movies['year'].fillna('2019', inplace = True)
df_movies['year'].astype(int)
#Llenar los meses con 01, osea 'Jan' y los días 1.
df_movies['month'].fillna('Jan', inplace = True)
df_movies['month'].astype(str)

df_movies['n_day'].fillna(1, inplace = True)
df_movies['n_day'].astype(int)

0        30
1        30
2        30
3        30
4        30
         ..
22993    20
22994     1
22995     1
22996    11
22997     2
Name: n_day, Length: 22998, dtype: int32

In [14]:
df_movies['date_added'] = (df_movies['year'].astype(str) + '-' + df_movies['month'] + '-' + df_movies['n_day'].astype(str)) #Ordenar las columnas de date_added.
df_movies['date_added'] = pd.to_datetime(df_movies['date_added']) #Convertir a formato de fecha.
df_movies.drop(['month', 'year', 'n_day'], axis = 1, inplace = True) #Eliminar columnas ya usadas.

Los nulos de rating se completan con 'G'.

In [15]:
df_movies['rating'].fillna('G', inplace = True) #Esta acción se realiza dos veces para reducir los NaN del df y poder trabajarlo sin problemas.

In [16]:
#Hacer un index a cada dato cruzado en rating, que deberia estar en duration, para que sea más facil ubicar y organizar en la columna correspondiente.
nex_duration = []
for ind, add in enumerate(df_movies['rating']):
    if add.__contains__('min'):
        new1 = (ind, add)
        nex_duration.append(new1)
    if add.__contains__('Season'):
        new2 = (ind, add)
        nex_duration.append(new2)

In [17]:
#Organizar los datos en la columna duration y eliminarlos de la columna rating, reemplazando por la 'G' para valores nulos de la columna.
for u in nex_duration:
    df_movies['duration'][u[0]] = u[1]
    df_movies['rating'][u[0]] = 'G'

C:\Users\Carlo\AppData\Local\Temp\ipykernel_13536\912807223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['duration'][u[0]] = u[1]
C:\Users\Carlo\AppData\Local\Temp\ipykernel_13536\912807223.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['rating'][u[0]] = 'G'


Para los nulos de la duracion, medir el promedio y la media de los eventos, ya sea de min o season. Es importante tener en cuenta que la relacion entre duration y type es:

- Movie se da siempre en min.
- TV show se da siempre en season.

Para continuar con las consigas y el proceso de ETL, poner en minúscula los datos de tipo texto, para que sea más fácil hacer una búsqueda del dato y manejo del mismo. Para ello: 

In [18]:
df_movies['type'] = df_movies['type'].str.lower()
df_movies['title'] = df_movies['title'].str.lower()
df_movies['cast'] = df_movies['cast'].str.lower()
df_movies['rating'] = df_movies['rating'].str.lower()
df_movies['duration'] = df_movies['duration'].str.lower()
df_movies['listed_in'] = df_movies['listed_in'].str.lower()

Separar la duración entre duration_int y duration_type.

In [19]:
duration_time = df_movies['duration'].str.split(expand=True)
duration_time.columns = ['duration_int', 'duration_type']
df_movies = pd.concat([df_movies, duration_time], axis = 1)

Los registros que hacen falta en la columna duration_type, son de min. Para ello, llenar con 'min' los NaN de duration_type y el promedio de 'min' para duration_int.

In [20]:
df_movies['duration_type'].fillna('min', inplace = True)
df_movies['duration_type'].astype(str)

0            min
1            min
2            min
3            min
4            min
          ...   
22993        min
22994    seasons
22995        min
22996        min
22997        min
Name: duration_type, Length: 22998, dtype: object

In [21]:
movies_type = df_movies[(df_movies['duration_type'] == 'min') & (df_movies['duration_int'] != None)] #Obtener los datos no nulos de las movies
movies_type = int(movies_type.duration_int.astype(float).mean()) #Promediar los datos anteriores
df_movies['duration_int'].fillna(movies_type, inplace = True) #Llenar los nulos con el promedio
df_movies['duration_int'] = df_movies['duration_int'].astype(int) #Convertir la columna a entero(int)

In [22]:
df_movies['duration_type'].replace('seasons', 'season', inplace = True) #Unificar el type de duration 'seasons' a 'season'.
df_movies.drop(columns = ['duration'], axis = 1, inplace = True) #Eliminar columna duration, ya que fue reemplazada por el int y type de duration.
df_movies = df_movies[['id', 'type', 'title', 'cast', 'date_added', 'release_year', 'rating', 'duration_int', 'duration_type', 'listed_in']]#Organizar df.

Ya tenemos el df de movies limpio. Para llevar a cabo las consultas, es necesario crear un dataframe para actores.

In [23]:
#Obtener cada uno de los actores de la columna df_movies['cast'], crear un diccionario con la plataforma y el año en donde aparece cada actor.
cast_general = df_movies['cast'].str.split(',')
dicc_actors = { 'actor' : [],
                'platform' : [],
                'year' : [] }
for ind, actors in enumerate(cast_general):
    for actor in actors:
        dicc_actors['actor'].append(actor)
        dicc_actors['platform'].append(df_movies['id'][ind])
        dicc_actors['year'].append(df_movies['release_year'][ind])
df_actors = pd.DataFrame(dicc_actors)
df_actors.drop(df_actors[df_actors['actor'] == 'not info'].index, inplace = True) # Eliminamos los valores nulos, ya que pueden afectar las consultas.

Ahora que ya se hizo uso de los datasets que tenían la información de las películas y los actores, convertirmos a csv para las consultas de más adelante.

In [24]:
df_movies.head()

,id,type,title,cast,date_added,release_year,rating,duration_int,duration_type,listed_in
0,as1,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2021-03-30,2014,g,113,min,"comedy, drama"
1,as2,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2021-03-30,2018,13+,110,min,"drama, international"
2,as3,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2021-03-30,2017,g,74,min,"action, drama, suspense"
3,as4,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2021-03-30,2014,g,69,min,documentary
4,as5,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",2021-03-30,1989,g,45,min,"drama, fantasy"


In [25]:
df_actors.head()

,actor,platform,year
0,brendan gleeson,as1,2014
1,taylor kitsch,as1,2014
2,gordon pinsent,as1,2014
3,mahesh manjrekar,as2,2018
4,abhay mahajan,as2,2018


In [26]:
df_movies.to_csv('Movies.csv', sep = ';', index = False)
df_actors.to_csv('Actors.csv', sep = ';', index = False)

Ahora trabajar sobre los ratings. Extraer dos dataframes y luego los unificarlos. Uno con el promedio de valoración por película, y el general de valoraciones por cada usuario.

In [26]:
df_rating1 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/1.csv', sep = ',')
df_rating2 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/2.csv', sep = ',')
df_rating3 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/3.csv', sep = ',')
df_rating4 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/4.csv', sep = ',')
df_rating5 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/5.csv', sep = ',')
df_rating6 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/6.csv', sep = ',')
df_rating7 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/7.csv', sep = ',')
df_rating8 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/8.csv', sep = ',')

In [27]:
#La columna timestamp contiene la fecha en un tipo entero(int). Cambiamos este formato a un date.
for ind, time in enumerate(df_rating1.timestamp):
    df_rating1['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating2.timestamp):
    df_rating2['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating3.timestamp):
    df_rating3['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating4.timestamp):
    df_rating4['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating5.timestamp):
    df_rating5['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating6.timestamp):
    df_rating6['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating7.timestamp):
    df_rating7['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating8.timestamp):
    df_rating8['timestamp'][ind] = date.fromtimestamp(time)

#Concatenar df para trabajar en conjunto. Este es el primer dataframe.
df_ratings = pd.concat([df_rating1, df_rating2, df_rating3, df_rating4, df_rating5, df_rating6, df_rating7, df_rating8], ignore_index=True).drop_duplicates()

#En caso de tener una máquina mejor, se puede realizar la transformación sobre el df general, sin tener que hacerlo por cada df de rating...
#for ind, time in enumerate(df_ratings.timestamp):
    #df_ratings['timestamp'][ind] = date.fromtimestamp(time)

C:\Users\Carlo\AppData\Local\Temp\ipykernel_13536\109220272.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating1['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_13536\109220272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating2['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_13536\109220272.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [28]:
#Extraer rating y el movieId del df de ratings general, para valorar el promedio por película.
rating = df_ratings[['rating']]
movies = df_ratings[['movieId']]

In [29]:
df_rating_mean_movie = pd.concat([rating, movies], axis = 1) #Concatenar las dos columnas creadas.
df_rating_mean_movie = df_rating_mean_movie.groupby(['movieId']).mean() #Agrupar por movieId para obtener el promedio. 
df_rating_mean_movie.reset_index(inplace = True) #Convertir movieId como columna. Es decir, resetear el index. Este es el segundo df.
df_rating_mean_movie.head()

,movieId,rating
0,as1,3.467131
1,as10,3.439571
2,as100,3.610723
3,as1000,3.557971
4,as1001,3.585288


In [30]:
df_ratings_total = pd.merge(df_ratings, df_rating_mean_movie, on = 'movieId') #Unir los dos df antes mencionados.

In [31]:
df_ratings_total.head()

,userId,rating_x,timestamp,movieId,rating_y
0,1,1.0,2015-03-09,as680,3.545952
1,583,4.5,2005-04-08,as680,3.545952
2,765,5.0,1998-03-11,as680,3.545952
3,2116,3.0,1996-07-01,as680,3.545952
4,2143,3.0,2015-06-23,as680,3.545952


Crear el df general(total) para realizar las consultas necesarias.

In [32]:
df_total = pd.merge(df_movies, df_ratings_total, left_on = 'id', right_on = 'movieId', how = 'inner') #Unimos los dos df de movies y ratings.

In [33]:
df_total_consultas = pd.DataFrame(df_total)
df_total_consultas.drop(columns = ['movieId', 'type', 'cast', 'date_added', 'rating', 'listed_in', 'userId', 'timestamp'], inplace = True) #Eliminar columna 'repetida'.
df_total_consultas.rename(columns = {'rating_x': 'score_user', 'rating_y' : 'score_mean'}, inplace = True) #Cambiar el nombre a las columnas.
df_total_consultas = df_total_consultas[['id', 'title', 'release_year', 'duration_int', 'duration_type', 'score_user', 'score_mean']] #Organizar df.
df_total_consultas = df_total_consultas.drop_duplicates() #Eliminar valores duplicados 

In [34]:
df_total_consultas.head()

,id,title,release_year,duration_int,duration_type,score_user,score_mean
0,as1,the grand seduction,2014,113,min,5.0,3.467131
1,as1,the grand seduction,2014,113,min,3.0,3.467131
3,as1,the grand seduction,2014,113,min,3.5,3.467131
4,as1,the grand seduction,2014,113,min,4.0,3.467131
5,as1,the grand seduction,2014,113,min,4.5,3.467131


In [41]:
df_total_consultas.to_csv('Total consultas.csv', sep = ';', index = False) #Exportar a csv.

https://fastapi-hbsx.onrender.com/docs

Ya tenemos los datasets limpios para hacer las consultas en nuestra fastAPI. Ahora, vamos a analizar los dataframe anteriormente creados para ML.

Vamos a usar el df_ratings para obtener la calificación de la movie en una fecha determinada, por el usuario. No exportamos el csv, por el peso que tendría, sino que, en el archivo del modelo concatenamos nuevamente los ratings

In [55]:
df_ratings.head()

,userId,rating,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500


Del df_movies, extraer las columnas id, title y listed_in para obtener la descripción de las mismas.  

In [75]:
df_movies.head()

,id,type,title,cast,date_added,release_year,rating,duration_int,duration_type,listed_in
0,as1,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2021-03-30,2014,g,113,min,"comedy, drama"
1,as2,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2021-03-30,2018,13+,110,min,"drama, international"
2,as3,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2021-03-30,2017,g,74,min,"action, drama, suspense"
3,as4,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2021-03-30,2014,g,69,min,documentary
4,as5,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",2021-03-30,1989,g,45,min,"drama, fantasy"


In [76]:
df_movies_description = df_movies[['id', 'title', 'listed_in']]

Usamos el df que tiene la calificación promedio de las películas.

In [84]:
df_rating_mean_movie.head()

,movieId,rating
0,as1,3.467131
1,as10,3.439571
2,as100,3.610723
3,as1000,3.557971
4,as1001,3.585288


In [85]:
df_total_ratings_movies = pd.merge(df_movies_description, df_rating_mean_movie, left_on = 'id', right_on = 'movieId', how = 'inner') #Unimos los df para tener información más concreta de la movie
df_total_ratings_movies.drop(columns = ['movieId'], axis = 1 ,inplace = True)

In [82]:
df_total_ratings_movies.to_csv('Ratings.csv', sep = ';', index = False) #Exportar a csv para trabajar el modelo.